In [ ]:
import pandas as pd
import numpy as np
from interpolate import *
from utils import *
from dataset import FinedustDataset
from model import FinedustLSTM

import torch
import torch.optim as optim
from torch.utils.data import DataLoader

In [ ]:
region="andong"
kma_path = f"../../collect_data/filtered/kma/merged/kma_{region}_meta.csv"
df = pd.read_csv(kma_path)
print_missing_info(df)

(40177, 21)
Series([], dtype: float64)
Index(['TM', 'STN', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'PV', 'WW',
       'CA_TOT', 'CA_MID', 'CT', 'VS', 'TS', 'IR', 'PM10', '지점', '위도', '경도'],
      dtype='object')


In [ ]:
# 학습을 위해 중국 데이터 append
df = concat_china_data(df)
df = convert_timesteps(df)

print_missing_info(df)

(8954, 107)
VV             0.033505
WD_yanan       0.022336
WS_yanan       0.022336
TA_yanan       0.100514
TD_yanan       0.245700
HM_yanan       0.703596
PS_yanan       0.033505
PR             0.055841
VV_tongliao    0.011168
WD_tongliao    0.011168
WS_tongliao    0.011168
TA_tongliao    0.145187
TD_tongliao    0.234532
HM_tongliao    0.256868
PS_tongliao    0.011168
PR_tongliao    0.011168
VV_qingdao     0.078177
WD_qingdao     0.089346
WS_qingdao     0.089346
TA_qingdao     0.078177
TD_qingdao     0.167523
HM_qingdao     0.792942
PS_qingdao     0.078177
PR_qingdao     0.078177
VV_chifeng     0.067009
WD_chifeng     0.022336
WS_chifeng     0.067009
TA_chifeng     0.100514
TD_chifeng     0.256868
HM_chifeng     0.189859
PS_chifeng     0.033505
PR_chifeng     0.022336
TA_dalian      0.044673
TD_dalian      0.189859
HM_dalian      0.033505
dtype: float64
Index(['TM', 'STN', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'PV',
       ...
       'TD_dalian', 'HM_dalian', 'PS_dalian', 'PT_dali

In [ ]:
df["diff"].value_counts(normalize=True) * 100

diff
3.0       93.500112
6.0        5.930310
9.0        0.189859
12.0       0.167523
15.0       0.067009
21.0       0.022336
75.0       0.011168
6744.0     0.011168
30.0       0.011168
852.0      0.011168
33.0       0.011168
27.0       0.011168
744.0      0.011168
24.0       0.011168
66.0       0.011168
18.0       0.011168
3678.0     0.011168
Name: proportion, dtype: float64

In [ ]:
dataset = FinedustDataset(df)
print(len(dataset))

837


* 시계열 데이터의 결측치가 6.5%이므로 반드시 interpolate해야 함

# 1. Linear interpolation

In [ ]:
df = simple_interpolate(df, method="linear")
df.shape

/home/han07301/ai_project/JH/LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')


(9684, 107)

In [ ]:
df["diff"].value_counts(normalize=True) * 100

diff
3.0       99.958690
6744.0     0.010327
852.0      0.010327
744.0      0.010327
3678.0     0.010327
Name: proportion, dtype: float64

In [ ]:
dataset = FinedustDataset(df)
print(len(dataset))

9124


* 837 -> 9124
* 일주일 단위 (window_size=56, prediction_length=56)일 때, 최대 데이터셋 길이는 9684 - 112 + 1 = 9573